# Extracting Features using pre-trained models and classification

Continuing the exploration about the concept of feature extraction, in this example we consider the 17 Flowers dataset. The method is the same that the previous example, we consider the VGG16 on the ImageNet, we store the features in the hdf5 file, finally, we train the classification model.

This is the first time we explore the 17 Flowers dataset, as the name suggests, we have 17 classes of flowers containing 80 images. The dataset is interesting because there is 17 class and not enough images. We are going to see the power of the transfer learning.

## Importing Libraries

In [1]:
# import the necessary packages
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
#compvis module
from compvis.io import HDF5DatasetWriter
from imutils import paths
import numpy as np
import progressbar
import argparse
import random
import os

## Setting the dateset

**Defining the paths for the dataset and the HDF5 file with the features**

In [2]:
DATA_PATH = "/home/igor/Documents/Artificial_Inteligence/Datasets/17flowers"
HDF5_PATH = "/home/igor/Documents/Artificial_Inteligence/Datasets/17flowers/hdf5/features.hdf5"

In [3]:
imagesPath = list(paths.list_images(DATA_PATH)) # making a list with all images

In [4]:
random.shuffle(imagesPath) # shuffling the images to have a better memory access

In [5]:
bs = 32 # defining the batch size

**Creating the list of labels**

We split the string that contains the path, for example, Data/17flowers/Class/Photo. We grab the class applying p.split(os.path.sep)[-2]. 

In [6]:
labels = [p.split(os.path.sep)[-2] for p in imagesPath]

**Encoding the labels**

In [7]:
le = LabelEncoder()
labels = le.fit_transform(labels)

## Defining the model for the feature extraction

The arguments for the VGG16 are weights and include_top. The first is "imagenet" and the second is False, we don't want to consider the fully connected layers, the classification will be made after by own model.

In [8]:
model = VGG16(weights = "imagenet", include_top = False)

**Building the dataset writer**

To build the dataset to store the features, we consider the class HDF5DatasetWriter. This class accepets 4 arguments dims (tuple with the number of raw and columns, in our case the number of images and the feature vector size, $7x7x512$), outputPath (the path to the hdf5 file), datakey (the name of the file), bufsize (buffer size by default 1000). The VGG16 at the end of the convolution layer returns 512 filters with size of $(7x7)$.

In [9]:
dataset = HDF5DatasetWriter((len(imagesPath), 512*7*7), HDF5_PATH, "features")

**Creating the list with the classes**, this function returns a list with names classes in the string format.

In [10]:
dataset.storeClassLabels(le.classes_)

### Feature Extraction

In [11]:
# Defining a progress bar
widgets = ["Extracting Features: ", progressbar.Percentage(), " ",
           progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=len(imagesPath), widgets=widgets).start()
# main loop over all images with a step size corresponding with the batch size
for i in np.arange(0, len(imagesPath), bs):
    
    batchPaths = imagesPath[i : i + bs] # list with the image pahts in the batch
    batchLabels = labels[i : i + bs]  # list with the labels in the batch
    batchImages = [] # empty list to store the image to the feature extraction
    
    #secondary loop to read and store the images in the batch size
    for (j, imgPath) in enumerate(batchPaths):
        image = load_img(imgPath, target_size=(224,224)) # reading and resizing the images
        image = img_to_array(image) # converting the image into an array
        image = np.expand_dims(image, axis=0) # expanding the dimensions to respect the channels
        image = imagenet_utils.preprocess_input(image) # preprocssing the image
        batchImages.append(image) # adding the current image into the image list
        
    batchImages = np.vstack(batchImages) # stacking the imgs
    features = model.predict(batchImages, batch_size=bs) # extracting the features in the batch
    features = features.reshape((features.shape[0], 512*7*7)) # resizing according with the hdf5 dataset5
    dataset.add(features, batchLabels) # adding the features and labels into the dataset
    pbar.update(i)
dataset.close()
pbar.finish()

Extracting Features: 100% |#####################################| Time: 0:00:16


# Training the model with the features

For this example, we consider the Logistic Regression classification model.

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import argparse
import pickle
import h5py

## Loading the HDF5 file with the features

In [13]:
dbt = h5py.File(HDF5_PATH, "r") # HDF5_PATH the file, r the read mode

**Defining the size of the training set**

As we have 3000 images, we consider $75\%$ of all images for the training set, totaling 2250.

In [14]:
i = int(dbt["labels"].shape[0]*0.75)

## Defining the classification model

We consider the Logistic Regression model using GridSearchCV that returns the best model, according with the hyperparemeters.

The hyperparemeters used in this example are $C$ the strictness and the solver.

In [15]:
# List of parameters
params = {"C": [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0], "solver" : ["newton-cg", "lbfgs"]}

In [16]:
# defining the model
model = GridSearchCV(LogisticRegression(max_iter=1000), params, cv = 5, n_jobs=1) # with cross validation equal to 5

In [17]:
# Fitting the model
model.fit(dbt["features"][:i], dbt["labels"][:i]) # [:i] we consider the training the staring from the index 0 into i

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000), n_jobs=1,
             param_grid={'C': [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0],
                         'solver': ['newton-cg', 'lbfgs']})

**The best parameters**

In [18]:
print("Best parameters {}".format(model.best_params_))

Best parameters {'C': 1.0, 'solver': 'lbfgs'}


**Predicting on the trained model**

In [19]:
predictions = model.predict(dbt["features"][i:]) #[i:] we consider the test set starting from i until the last index

**Evaluating the model with the classification report**

In [20]:
target_names = ['cats', 'dogs', 'pandas'] # creating the target list

In [21]:
cr = classification_report(dbt["labels"][i:], predictions)

In [22]:
print(cr)

              precision    recall  f1-score   support

           0       0.92      0.50      0.65        22
           1       0.91      0.91      0.91        22
           2       0.94      0.94      0.94        16
           3       1.00      0.88      0.93        16
           4       1.00      0.90      0.95        21
           5       0.60      0.68      0.64        22
           6       0.56      0.95      0.71        19
           7       0.96      1.00      0.98        24
           8       1.00      0.83      0.90        23
           9       0.92      1.00      0.96        22
          10       0.86      1.00      0.93        19
          11       1.00      1.00      1.00        14
          12       1.00      0.89      0.94        19
          13       1.00      1.00      1.00        26
          14       0.91      0.88      0.89        24
          15       1.00      0.94      0.97        17
          16       1.00      1.00      1.00        14

    accuracy              

# Conclusions

As is the first time that this dataset is used in this project, we do not have As is the first time that this dataset is used in this project, we do not have a point of comparison. The fact is, the result of the accuracy on the test set is considerable, $91\%$. Again, transfer learning enabled us to obtain a good classification model.